# Fetching Data from MySQL into a Pandas DataFrame

## Setup

Before we begin, you need to ensure that Python is installed on your system. Additionally, we'll be using the `mysql-connector-python` package to connect to our MySQL database and the `pandas` package to work with DataFrames. Install these packages using the following commands:

```bash
pip install mysql-connector-python
```

With the installations out of the way, let’s set up our working environment.

In [2]:
# install mysql connector
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# import libraries
import mysql.connector
from mysql.connector import Error

## Establishing a Connection with MySQL Database

To work with a MySQL database from Python, we need to establish a connection. MySQL Connector/Python is a driver that facilitates this. After we import the required module, then we can define a function to handle the database connection.

Replace `host_name`, `user_name`, `user_password`, and `db_name` with our MySQL server’s details.

In [7]:
# function to open connection to db
def connect_to_db(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [12]:
# define mysql details
host   = 'localhost'
user   = 'root'
passwd = 'Aldi.Fauzan12345'
databs = 'sakila'

In [13]:
# try to connect to sakila database
connect_to_db(host, user, passwd, databs)

MySQL Database connection successful


## Writing the Function to Fetch Data

Now, let’s create a function that will use this connection to fetch data and return a pandas DataFrame.

In [14]:
# import pandas
import pandas as pd

# import warning to ignore warning
import warnings
warnings.filterwarnings('ignore')

In [16]:
# function to fetch data from database, the connection is closed after use.
def fetch_data(host_name, user_name, user_password, db_name, query, params=None):
    try:
        # use with statement (context manager) to ensure the connection is closed after use
        with mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        ) as connection:
            # read the query to dataframe format
            data = pd.read_sql(query, connection, params=params)
            return data

    except Error as e:
        print(f"Error while connecting to MySQL or fetching data: {e}")
        return None

Above function uses `pd.read_sql()` from pandas, which runs the SQL query through the provided connection and returns a DataFrame. If you need to pass parameters to your query, you can do so by passing a tuple as the `params` argument.

In [17]:
# usage example (the mysql detail already defined)
# simple query
query_actor = """

select * from actor;

"""

In [18]:
# fetch data using function
df = fetch_data(host, user, passwd, databs, query_actor)

In [19]:
# show data
df.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [20]:
# show info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   actor_id     200 non-null    int64         
 1   first_name   200 non-null    object        
 2   last_name    200 non-null    object        
 3   last_update  200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 6.4+ KB


We can use `SQLAlchemy`, which is another popular Python library to do this job. Using SQLAlchemy can be beneficial for its ORM capabilities and flexibility (simpler code).

In [21]:
# install sqlalchemy
!pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


In [22]:
# import sqlalchemy engine
import sqlalchemy
from sqlalchemy import create_engine

In [24]:
# function to fetch data using sqlalchemy engine
def alchemy_fetch_data(host_name, user_name, user_password, db_name, query, params=None):
    try:
        # format the database URL
        database_url = f"mysql+mysqlconnector://{user_name}:{user_password}@{host_name}/{db_name}"

        # create a database engine
        engine = create_engine(database_url)

        # fetch data into a pandas df
        data = pd.read_sql_query(query, engine, params=params)
        return data

    except Exception as e:
        print(f"Error: {e}")
        return None

    finally:
        # dispose the engine
        engine.dispose()

In [25]:
# usage example
# simple query with cte
query_actor_cte = """

with films as (
    select film_id, title, rating, actor_id
    from film
    left join film_actor fa using (film_id)
), actors as (
    select actor_id, first_name, last_name
    from actor
)
select *
from films
join actors using (actor_id)

"""

In [26]:
# fetch data using sqlalchemy
df2 = alchemy_fetch_data(host, user, passwd, databs, query_actor_cte)

In [29]:
# show data
df2.head()

,actor_id,film_id,title,rating,first_name,last_name
0,1,1,ACADEMY DINOSAUR,PG,PENELOPE,GUINESS
1,10,1,ACADEMY DINOSAUR,PG,CHRISTIAN,GABLE
2,20,1,ACADEMY DINOSAUR,PG,LUCILLE,TRACY
3,30,1,ACADEMY DINOSAUR,PG,SANDRA,PECK
4,40,1,ACADEMY DINOSAUR,PG,JOHNNY,CAGE


In [ ]:
# show info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5462 entries, 0 to 5461
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   actor_id    5462 non-null   int64 
 1   film_id     5462 non-null   int64 
 2   title       5462 non-null   object
 3   rating      5462 non-null   object
 4   first_name  5462 non-null   object
 5   last_name   5462 non-null   object
dtypes: int64(2), object(4)
memory usage: 256.2+ KB


## Best Practices

When working with databases, always ensure that you manage connections carefully. Connections should be opened as late as possible and closed as soon as you're done with them. In the provided functions, we manually close the connection after we're done with our operations.

Always validate and sanitize any input that comes from users to protect against SQL injection when building queries. Using parameterized queries, is one of the best practices to mitigate such risks.